In [1]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

In [2]:
def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

In [3]:
def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size

In [4]:
def mask(image):
    r = (image==255)
    return r


In [5]:

def get_pixel_else_0(l, idx, idy, default=0):
    try:
        return l[idx,idy]
    except IndexError:
        return default

    
def si(x,threshold):
    if x>threshold:
        return 1
    else:
        return 0

In [28]:
def CS_LBP(img,th,encoding):
    
    cslbp = np.zeros_like(img,dtype=np.float64)
    for x in range(0, len(cslbp)):
        for y in range(0, len(cslbp[0])):
            top_left      = get_pixel_else_0(img, x-1, y-1)
            top_up        = get_pixel_else_0(img, x, y-1)
            top_right     = get_pixel_else_0(img, x+1, y-1)
            right         = get_pixel_else_0(img, x+1, y )
            left          = get_pixel_else_0(img, x-1, y )
            bottom_left   = get_pixel_else_0(img, x-1, y+1)
            bottom_right  = get_pixel_else_0(img, x+1, y+1)
            bottom_down   = get_pixel_else_0(img, x,   y+1 )

            v1 = si(int(right)-int(left),th)
            v2 = si(int(bottom_right)-int(top_left),th)
            v3 = si(int(bottom_down)-int(top_up),th)
            v4 = si(int(bottom_left)-int(top_right),th)
            
            va = [v1,v2,v3,v4]
            weights = [1, 2, 4, 8]
            res = 0
            for a in range(0, len(weights)):
                res += weights[a] * va[a]
            
            if(encoding == 'ri'):
                if(res==2 or res==4 or res==8):
                    res = 1
                elif(res==6 or res ==9 or res==12):
                    res=3
                elif(res==10):
                    res=5
                elif(res==11 or res==13 or res==14):
                    res=7    
            cslbp.itemset((x,y), res)

    return cslbp

In [29]:
def CenterSymmetric_LBP(images,dim,sizes,threshold = 5,encoding='ri'):
    hist = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    print(k,i,j)
                    cslbp_pens=CS_LBP(images['c'+str(k)+'_'+i+'_'+j][:,:,dim],threshold,encoding)
                    cslbp_pens+=mask(images['c'+str(k)+'_'+i+'_'+j][:,:,0])*100

                    h,_ = np.histogram(cslbp_pens.ravel(),range(0,17))
                    s = sizes['c'+str(k)+'_'+i+'_'+j]
                    h = h/s
                    if(encoding=='ri'):
                        hi=[]
                        for l in [0, 1, 3, 5, 7, 15]:    
                            hi.append(h[l])
                        hist['c'+str(k)+'_'+i+'_'+j] = np.array(hi)
                    else:
                        hist['c'+str(k)+'_'+i+'_'+j]=np.array(h)
                except:
                    break
                   
    return hist

In [22]:
def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def compute_dist(param1,param2,n1,n2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def distance_distribution(features,sizes):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' same {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m],sizes['c'+str(k)+'_'+i+'_'+j],sizes['c'+str(k)+'_'+n+'_'+m]))
                                #print(' diff {}   :{}'.format('c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m,diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m]))
                        except:
                            
                            pass
    print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [8]:
images = load_images()
sizes = n_size(images)

# using ri encoding

In [16]:
hist_Y = CenterSymmetric_LBP(images,0,sizes,20)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [18]:
hist_Cb = CenterSymmetric_LBP(images,1,sizes,10)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [19]:
hist_Cr = CenterSymmetric_LBP(images,2,sizes,10)

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [20]:
hist_Y['c1_1_2']

array([0.34667571, 0.20624152, 0.1275441 , 0.00644505, 0.21438263,
       0.09871099])

In [23]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

Lengths same pens-1465 different pens-1456


/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
16.78427760655271 28.841758031628505 20.954441919733494


In [24]:
Y = pd.DataFrame(hist_Y)
Cb = pd.DataFrame(hist_Cb)
Cr = pd.DataFrame(hist_Cr)

In [25]:
all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CSLBP_ri_T=20,10,10.csv',sep = ',')

# using default encoding

In [30]:
hist_Y = CenterSymmetric_LBP(images,0,sizes,20,encoding='default')

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

In [32]:
hist_Cb = CenterSymmetric_LBP(images,1,sizes,10,encoding='default')
hist_Cr = CenterSymmetric_LBP(images,2,sizes,10,encoding='default')

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 2 1
2 2 2
2 2 3
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 1 6
3 2 1
3 2 2
3 2 3
3 2 4
4 1 1
4 1 2
4 1 3
4 1 4
4 1 5
4 1 6
4 1 7
4 1 8
4 2 1
4 2 2
4 2 3
5 1 1
5 1 2
5 1 3
5 1 4
5 1 5
5 1 6
5 1 7
5 2 1
5 2 2
5 2 3
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 1 6
6 1 7
6 2 1
6 2 2
6 2 3
7 1 1
7 1 2
7 1 3
7 1 4
7 1 5
7 1 6
7 2 1
7 2 2
7 2 3
8 1 1
8 1 2
8 1 3
8 1 4
8 1 5
8 1 6
8 1 7
8 1 8
8 2 1
8 2 2
8 2 3
9 1 1
9 1 2
9 1 3
9 1 4
9 1 5
9 1 6
9 1 7
9 1 8
9 2 1
9 2 2
9 2 3
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 2 1
10 2 2
10 2 3
10 2 4
11 1 1
11 1 2
11 1 3
11 1 4
11 1 5
11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1

11 1 6
11 1 7
11 1 8
11 2 1
11 2 2
11 2 3
12 1 1
12 1 2
12 1 3
12 1 4
12 1 5
12 1 6
12 1 7
12 2 1
12 2 2
12 2 3
13 1 1
13 1 2
13 1 3
13 1 4
13 1 5
13 2 1
13 2 2
13 2 3
13 2 4
14 1 1
14 1 2
14 1 3
14 1 4
14 1 5
14 1 6
14 1 7
14 2 1
14 2 2
14 2 3
15 1 1
15 1 2
15 1 3
15 1 4
15 1 5
15 2 1
15 2 2
15 2 3
16 1 1
16 1 2
16 1 3
16 1 4
16 1 5
16 1 6
16 1 7
16 2 1
16 2 2
16 2 3
17 1 1
17 1 2
17 1 3
17 1 4
17 1 5
17 1 6
17 2 1
17 2 2
17 2 3
18 1 1
18 1 2
18 1 3
18 1 4
18 1 5
18 1 6
18 1 7
18 2 1
18 2 2
18 2 3
18 2 4
19 1 1
19 1 2
19 1 3
19 1 4
19 1 5
19 1 6
19 2 1
19 2 2
19 2 3
20 1 1
20 1 2
20 1 3
20 1 4
20 1 5
20 1 6
20 1 7
20 2 1
20 2 2
20 2 3
21 1 1
21 1 2
21 1 3
21 1 4
21 2 1
21 2 2
21 2 3
22 1 1
22 1 2
22 1 3
22 1 4
22 1 5
22 1 6
22 1 7
22 2 1
22 2 2
22 2 3
22 2 4
22 2 5
23 1 1
23 1 2
23 1 3
23 1 4
23 1 5
23 1 6
23 1 7
23 2 1
23 2 2
23 2 3
24 1 1
24 1 2
24 1 3
24 1 4
24 1 5
24 2 1
24 2 2
24 2 3
25 1 1
25 1 2
25 1 3
25 1 4
25 1 5
25 1 6
25 1 7
25 1 8
25 2 1
25 2 2
25 2 3
25 2 4
26 1 1
26 1 2

In [33]:
same_pens_Y,diff_pens_Y = distance_distribution(hist_Y,sizes)
same_pens_Y = [ v for v in same_pens_Y. values() ]
diff_pens_Y = [ v for v in diff_pens_Y. values() ]

same_pens_Cb,diff_pens_Cb = distance_distribution(hist_Cb,sizes)
same_pens_Cb = [ v for v in same_pens_Cb. values() ]
diff_pens_Cb = [ v for v in diff_pens_Cb. values() ]

same_pens_Cr,diff_pens_Cr = distance_distribution(hist_Cr,sizes)
same_pens_Cr = [ v for v in same_pens_Cr. values() ]
diff_pens_Cr = [ v for v in diff_pens_Cr. values() ]

dy=dist(same_pens_Y,diff_pens_Y,1465,1456)
dcb = dist(same_pens_Cb,diff_pens_Cb,1465,1456)
dcr = dist(same_pens_Cr,diff_pens_Cr,1465,1456)

print(dy,dcb,dcr)

/home/mohit/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
Lengths same pens-1465 different pens-1456
18.95091789816756 30.67709874436664 23.74617738259606


In [34]:
Y = pd.DataFrame(hist_Y)
Cb = pd.DataFrame(hist_Cb)
Cr = pd.DataFrame(hist_Cr)

In [35]:
all_features = pd.concat([Y,Cb,Cr])
all_features.to_csv('/home/mohit/Desktop/featuresUpdated/CSLBP_default_T=20,10,10.csv',sep = ',')